## Тестовое задание для bewise.ai
#### Фамилия, имя: Демьянов Иван


### Загрузка и предобработкаданных

In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from manager_names import name_recognize_nltk, make_names_list, get_name_and_replic

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/ivandemyanov/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
test_data = pd.read_csv('./test_data.csv')
df = test_data.copy(deep=True)
test_data.head()

,dlg_id,line_n,role,text
0,0,0,client,Алло
1,0,1,manager,Алло здравствуйте
2,0,2,client,Добрый день
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...
4,0,4,client,Ага


In [3]:
test_data.text[3]

'Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается'

In [4]:
nltk.download('stopwords')
STOPWORDS = nltk.corpus.stopwords.words('russian')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ivandemyanov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
def delete_stopwords(text):
    """
    :param str text: Input text 
    :return List[str]: List of words
    """
    text = text.lower()
    text = re.sub(r'[^А-Яа-я ]', '', text).split(' ')
    text = [w for w in text if w not in set(STOPWORDS) and w != '']
    return " ".join(text)

In [6]:
delete_stopwords(test_data.text[3])

'зовут ангелина компания диджитал бизнес звоним поводу продления лицензии серым скоро срок заканчивается'

In [7]:
df['new_text'] = df.text.apply(delete_stopwords)
df

,dlg_id,line_n,role,text,new_text
0,0,0,client,Алло,алло
1,0,1,manager,Алло здравствуйте,алло здравствуйте
2,0,2,client,Добрый день,добрый день
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,зовут ангелина компания диджитал бизнес звоним...
4,0,4,client,Ага,ага
...,...,...,...,...,...
475,5,138,manager,По поводу виджетов и с ними уже обсудите конкр...,поводу виджетов ними обсудите конкретно продам
476,5,139,manager,Все я вам высылаю счет и с вами на связи если ...,высылаю счет вами связи будут вопросы можете п...
477,5,140,client,Спасибо спасибо,спасибо спасибо
478,5,141,client,Да да тогда созвонимся ага спасибо вам давайте,созвонимся ага спасибо давайте


### Извлечение реплик, где менеджер представил себя, и имени менеджера

In [8]:
df['NLTK'] = df.text.apply(name_recognize_nltk)
df

,dlg_id,line_n,role,text,new_text,NLTK
0,0,0,client,Алло,алло,0
1,0,1,manager,Алло здравствуйте,алло здравствуйте,0
2,0,2,client,Добрый день,добрый день,0
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,зовут ангелина компания диджитал бизнес звоним...,"[[ангелина, ангелина, 1.0, Меня зовут ангелина..."
4,0,4,client,Ага,ага,0
...,...,...,...,...,...,...
475,5,138,manager,По поводу виджетов и с ними уже обсудите конкр...,поводу виджетов ними обсудите конкретно продам,0
476,5,139,manager,Все я вам высылаю счет и с вами на связи если ...,высылаю счет вами связи будут вопросы можете п...,0
477,5,140,client,Спасибо спасибо,спасибо спасибо,0
478,5,141,client,Да да тогда созвонимся ага спасибо вам давайте,созвонимся ага спасибо давайте,0


In [9]:
names, replics = get_name_and_replic(df)
df['name'] = '_'
df['introduce'] = False

def write_manager_names(text):
    
    for n, r in zip(names, replics):   
        if r == text:
            return n
    return '_'

def manager_introduced_r(text):

    for r in replics:   
        if r == text:
            return True
    return False

df['name'] = df.new_text.apply(write_manager_names)
df['introduce'] = df.new_text.apply(manager_introduced_r)
df

,dlg_id,line_n,role,text,new_text,NLTK,name,introduce
0,0,0,client,Алло,алло,0,_,False
1,0,1,manager,Алло здравствуйте,алло здравствуйте,0,_,False
2,0,2,client,Добрый день,добрый день,0,_,False
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,зовут ангелина компания диджитал бизнес звоним...,"[[ангелина, ангелина, 1.0, Меня зовут ангелина...",_,False
4,0,4,client,Ага,ага,0,_,False
...,...,...,...,...,...,...,...,...
475,5,138,manager,По поводу виджетов и с ними уже обсудите конкр...,поводу виджетов ними обсудите конкретно продам,0,_,False
476,5,139,manager,Все я вам высылаю счет и с вами на связи если ...,высылаю счет вами связи будут вопросы можете п...,0,_,False
477,5,140,client,Спасибо спасибо,спасибо спасибо,0,_,False
478,5,141,client,Да да тогда созвонимся ага спасибо вам давайте,созвонимся ага спасибо давайте,0,_,False


In [10]:
d = {"dlg_id":np.arange(len(names)), "names": names, "intro_replics": replics}
res_frame = pd.DataFrame(d, index=np.arange(len(names)))
res_frame

,dlg_id,names,intro_replics
0,0,ангелина,Меня зовут ангелина компания диджитал бизнес з...
1,1,ангелина,Меня зовут ангелина компания диджитал бизнес з...
2,2,ангелина,Меня зовут ангелина компания диджитал бизнес з...
3,3,максим,Добрый меня максим зовут компания китобизнес у...
4,4,имя не было указано,"остутсвуют реплики, где менеджер представил себя"
5,5,анастасия,Да это анастасия


### Извлечение названий компаний

In [11]:
from keybert import KeyBERT
from sentence_transformers import SentenceTransformer

sentence_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')
kw_model = KeyBERT(model=sentence_model)
keywords = kw_model.extract_keywords(test_data.text)
df['company name'] = '_'

def find_comp(text):

    lst = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 3), stop_words=None,
                              use_mmr=True, diversity=0.2)
    for l in lst:
        if 'компания' in l[0] and l[1] < 0.92:
            return l[0]
    return '_'

frame_list = []
for idx in np.unique(df.dlg_id):
    frame_list.append(df[df.dlg_id == idx][df.role == 'manager'].head(5).new_text.apply(find_comp))
company_frame = pd.concat(frame_list)
df['company name'] = company_frame
df['company name'].fillna('_', inplace=True)
df

,dlg_id,line_n,role,text,new_text,NLTK,name,introduce,company name
0,0,0,client,Алло,алло,0,_,False,_
1,0,1,manager,Алло здравствуйте,алло здравствуйте,0,_,False,_
2,0,2,client,Добрый день,добрый день,0,_,False,_
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,зовут ангелина компания диджитал бизнес звоним...,"[[ангелина, ангелина, 1.0, Меня зовут ангелина...",_,False,компания диджитал бизнес
4,0,4,client,Ага,ага,0,_,False,_
...,...,...,...,...,...,...,...,...,...
475,5,138,manager,По поводу виджетов и с ними уже обсудите конкр...,поводу виджетов ними обсудите конкретно продам,0,_,False,_
476,5,139,manager,Все я вам высылаю счет и с вами на связи если ...,высылаю счет вами связи будут вопросы можете п...,0,_,False,_
477,5,140,client,Спасибо спасибо,спасибо спасибо,0,_,False,_
478,5,141,client,Да да тогда созвонимся ага спасибо вам давайте,созвонимся ага спасибо давайте,0,_,False,_


In [12]:
dlg_ids = list(df[df['company name'] != '_'][['dlg_id', 'company name']].dlg_id)
company_names = list(df[df['company name'] != '_'][['dlg_id', 'company name']]['company name'])
d = dict(zip(dlg_ids, company_names))
res_frame['company names'] = '_'

def write_company_names(dlg_id):
    
    if dlg_id  in d.keys():
        return d[dlg_id]
    else:
        return '_'

res_frame['company names'] = res_frame.dlg_id.apply(write_company_names)
res_frame

,dlg_id,names,intro_replics,company names
0,0,ангелина,Меня зовут ангелина компания диджитал бизнес з...,компания диджитал бизнес
1,1,ангелина,Меня зовут ангелина компания диджитал бизнес з...,компания диджитал бизнес
2,2,ангелина,Меня зовут ангелина компания диджитал бизнес з...,компания диджитал бизнес
3,3,максим,Добрый меня максим зовут компания китобизнес у...,компания китобизнес
4,4,имя не было указано,"остутсвуют реплики, где менеджер представил себя",_
5,5,анастасия,Да это анастасия,_


### Извлечение реплик с приветсвием/прощанием

#### Извлечение реплик с приветсвием

In [13]:
greetings_list = ['здравствуйте', 'доброе утро', 'добрый день', 'добрый вечер', 'приветствую вас']

def find_greetings(text):

    for gr in greetings_list:
        if gr in text:
            return True
    return False

greetings_frame_list = []
for idx in np.unique(df.dlg_id):
    greetings_frame_list.append(df[df.dlg_id == idx][df.role == 'manager'].head(5).new_text.apply(find_greetings))
greetings_frame = pd.concat(greetings_frame_list)
df['greetings'] = greetings_frame
df['greetings'].fillna(False, inplace=True)
df

,dlg_id,line_n,role,text,new_text,NLTK,name,introduce,company name,greetings
0,0,0,client,Алло,алло,0,_,False,_,False
1,0,1,manager,Алло здравствуйте,алло здравствуйте,0,_,False,_,True
2,0,2,client,Добрый день,добрый день,0,_,False,_,False
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,зовут ангелина компания диджитал бизнес звоним...,"[[ангелина, ангелина, 1.0, Меня зовут ангелина...",_,False,компания диджитал бизнес,False
4,0,4,client,Ага,ага,0,_,False,_,False
...,...,...,...,...,...,...,...,...,...,...
475,5,138,manager,По поводу виджетов и с ними уже обсудите конкр...,поводу виджетов ними обсудите конкретно продам,0,_,False,_,False
476,5,139,manager,Все я вам высылаю счет и с вами на связи если ...,высылаю счет вами связи будут вопросы можете п...,0,_,False,_,False
477,5,140,client,Спасибо спасибо,спасибо спасибо,0,_,False,_,False
478,5,141,client,Да да тогда созвонимся ага спасибо вам давайте,созвонимся ага спасибо давайте,0,_,False,_,False


In [14]:
dlg_ids = list(df[df['greetings'] == True][['dlg_id', 'text']].dlg_id)
greetings_repl = list(df[df['greetings'] == True][['dlg_id', 'text']]['text'])
d = dict(zip(dlg_ids, greetings_repl))
res_frame['greetings'] = '_'
res_frame

def write_greetings(dlg_id):
    
    if dlg_id  in d.keys():
        return d[dlg_id]
    else:
        return '_'
    
res_frame['greetings'] = res_frame.dlg_id.apply(write_greetings)
res_frame

,dlg_id,names,intro_replics,company names,greetings
0,0,ангелина,Меня зовут ангелина компания диджитал бизнес з...,компания диджитал бизнес,Алло здравствуйте
1,1,ангелина,Меня зовут ангелина компания диджитал бизнес з...,компания диджитал бизнес,Алло здравствуйте
2,2,ангелина,Меня зовут ангелина компания диджитал бизнес з...,компания диджитал бизнес,Алло здравствуйте
3,3,максим,Добрый меня максим зовут компания китобизнес у...,компания китобизнес,Алло дмитрий добрый день
4,4,имя не было указано,"остутсвуют реплики, где менеджер представил себя",_,_
5,5,анастасия,Да это анастасия,_,_


#### Извлечение реплик с прощанием

In [15]:
farewells_list = ['До свидания', 'Всего хорошего', 'до свидания', 'всего хорошего'
            'До понедельника', 'До вторника', 'До среды', 'До четверга', 'До пятницы',
            'до понедельника', 'до вторника', 'до среды', 'до четверга', 'до пятницы']

def find_farewells(text):

    for fw in farewells_list:
        if fw in text:
            return True
    return False

farewells_frame_list = []
for idx in np.unique(df.dlg_id):
    farewells_frame_list.append(df[df.dlg_id == idx][df.role == 'manager'].tail(5).text.apply(find_farewells))
farewells_frame = pd.concat(farewells_frame_list)
df['farewells'] = farewells_frame
df['farewells'].fillna(False, inplace=True)
df

,dlg_id,line_n,role,text,new_text,NLTK,name,introduce,company name,greetings,farewells
0,0,0,client,Алло,алло,0,_,False,_,False,False
1,0,1,manager,Алло здравствуйте,алло здравствуйте,0,_,False,_,True,False
2,0,2,client,Добрый день,добрый день,0,_,False,_,False,False
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,зовут ангелина компания диджитал бизнес звоним...,"[[ангелина, ангелина, 1.0, Меня зовут ангелина...",_,False,компания диджитал бизнес,False,False
4,0,4,client,Ага,ага,0,_,False,_,False,False
...,...,...,...,...,...,...,...,...,...,...,...
475,5,138,manager,По поводу виджетов и с ними уже обсудите конкр...,поводу виджетов ними обсудите конкретно продам,0,_,False,_,False,False
476,5,139,manager,Все я вам высылаю счет и с вами на связи если ...,высылаю счет вами связи будут вопросы можете п...,0,_,False,_,False,False
477,5,140,client,Спасибо спасибо,спасибо спасибо,0,_,False,_,False,False
478,5,141,client,Да да тогда созвонимся ага спасибо вам давайте,созвонимся ага спасибо давайте,0,_,False,_,False,False


In [16]:
dlg_ids = list(df[df['farewells'] == True][['dlg_id', 'text']].dlg_id)
farewells_repl = list(df[df['farewells'] == True][['dlg_id', 'text']]['text'])
d = dict(zip(dlg_ids, farewells_repl))
res_frame['farewells'] = '_'

def write_farewells(dlg_id):
    
    if dlg_id  in d.keys():
        return d[dlg_id]
    else:
        return '_'

res_frame['farewells'] = res_frame.dlg_id.apply(write_farewells)
res_frame

,dlg_id,names,intro_replics,company names,greetings,farewells
0,0,ангелина,Меня зовут ангелина компания диджитал бизнес з...,компания диджитал бизнес,Алло здравствуйте,Всего хорошего до свидания
1,1,ангелина,Меня зовут ангелина компания диджитал бизнес з...,компания диджитал бизнес,Алло здравствуйте,До свидания
2,2,ангелина,Меня зовут ангелина компания диджитал бизнес з...,компания диджитал бизнес,Алло здравствуйте,_
3,3,максим,Добрый меня максим зовут компания китобизнес у...,компания китобизнес,Алло дмитрий добрый день,_
4,4,имя не было указано,"остутсвуют реплики, где менеджер представил себя",_,_,Во вторник все ну с вами да тогда до вторника ...
5,5,анастасия,Да это анастасия,_,_,Ну до свидания хорошего вечера


### Проверка требования к менеджеру (f) и результаты

In [17]:
res_frame['politeness_requirement'] = res_frame.apply(lambda row: '_' not in (row["greetings"] + row["farewells"]), axis=1)
res_frame

,dlg_id,names,intro_replics,company names,greetings,farewells,politeness_requirement
0,0,ангелина,Меня зовут ангелина компания диджитал бизнес з...,компания диджитал бизнес,Алло здравствуйте,Всего хорошего до свидания,True
1,1,ангелина,Меня зовут ангелина компания диджитал бизнес з...,компания диджитал бизнес,Алло здравствуйте,До свидания,True
2,2,ангелина,Меня зовут ангелина компания диджитал бизнес з...,компания диджитал бизнес,Алло здравствуйте,_,False
3,3,максим,Добрый меня максим зовут компания китобизнес у...,компания китобизнес,Алло дмитрий добрый день,_,False
4,4,имя не было указано,"остутсвуют реплики, где менеджер представил себя",_,_,Во вторник все ну с вами да тогда до вторника ...,False
5,5,анастасия,Да это анастасия,_,_,Ну до свидания хорошего вечера,False


In [18]:
df

,dlg_id,line_n,role,text,new_text,NLTK,name,introduce,company name,greetings,farewells
0,0,0,client,Алло,алло,0,_,False,_,False,False
1,0,1,manager,Алло здравствуйте,алло здравствуйте,0,_,False,_,True,False
2,0,2,client,Добрый день,добрый день,0,_,False,_,False,False
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...,зовут ангелина компания диджитал бизнес звоним...,"[[ангелина, ангелина, 1.0, Меня зовут ангелина...",_,False,компания диджитал бизнес,False,False
4,0,4,client,Ага,ага,0,_,False,_,False,False
...,...,...,...,...,...,...,...,...,...,...,...
475,5,138,manager,По поводу виджетов и с ними уже обсудите конкр...,поводу виджетов ними обсудите конкретно продам,0,_,False,_,False,False
476,5,139,manager,Все я вам высылаю счет и с вами на связи если ...,высылаю счет вами связи будут вопросы можете п...,0,_,False,_,False,False
477,5,140,client,Спасибо спасибо,спасибо спасибо,0,_,False,_,False,False
478,5,141,client,Да да тогда созвонимся ага спасибо вам давайте,созвонимся ага спасибо давайте,0,_,False,_,False,False
